In [1]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5

In [4]:
import datetime

In [5]:
def fget_data(currency):
    mt5.initialize()
    ohlc_data = pd.DataFrame(mt5.copy_rates_range(currency, mt5.TIMEFRAME_M15, datetime.datetime(2023, 1, 1), datetime.datetime.now()))
    ohlc_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    ohlc_data.dropna(inplace=True)

    return ohlc_data

In [6]:
nowd=fget_data('GOLD')
nowd['mtime']=pd.to_datetime(nowd['time'],unit='s')
nowd = nowd[nowd['mtime'].dt.date != pd.to_datetime('2015-01-02').date()]
nowd.reset_index(drop=True, inplace=True)
using_data=nowd


In [7]:
from functions import calc_prof

In [3]:
def x_points(data,index):
    # index=index.tolist()[0]
    val_h=data.iloc[index]
    start_date = val_h['mtime'].date()
    t=val_h['mtime'].time()
    filtered_data = data[(data['mtime'].dt.date == start_date) & (data.index <= index)]
    # print(filtered_data)
    # print(f'filtered_data:\n{filtered_data.head()}')
    max_value = filtered_data['Close'].max()
    min_value = filtered_data['Close'].min()
    return max_value, min_value
    
    
def distance_checker(price,value,lim):
    plh=price-value
    plh=abs(plh)
    if plh >lim:
        return 1
    else:
        return 0
    
def find_status(data,index,window,size):
    cspreads=[]
    # print(index,window,size)
    # print("fstatus here1")
    for i in range(window):
        datapoint=data.iloc[index-i]
        close=datapoint['Close']
        openv=datapoint['Open']
        cspread=abs(close-openv)
        cspreads.append(cspread)
    # print("fstatus here2")
    if max(cspreads)>size:
        return 1
    else:
        return 0

In [8]:
def daily(data,adjtime,lim,window,c_size,sl_adj,multiplier,cash):
    trades=[]
    trade=0
    opendate=None
    openprice=0
    tp=0
    sl=0
    cash=cash
    distcounter=0
    statuscounter=0
    distcounter2=0
    statuscounter2=0
    for i in range(len(data)):
        val_h=data.loc[i]
        time=val_h['mtime'].time()
        adj_time = pd.to_datetime(adjtime).time()# adtime being such astring '03:00:00'
        if time<adj_time:
            continue
        else:
            
            tday=val_h['mtime'].date()
            target_datetime = pd.to_datetime(f'{tday} {adj_time}')
            # print(target_datetime)
            val=data[data['mtime'] == target_datetime]
            # print(val)
            # print(val.index)
            
            startindex=val.index
            ind=startindex.tolist()[0]
            # print(ind)
            begin_max_value,begin_min_value=x_points(data,ind)
            maxv=begin_max_value
            # print(maxv)
            minv=begin_min_value
            # print(minv)
            
            # print(f'tade{trade}')
            price=val_h['Close']
            if trade==0:
                if price>maxv:
                    if distance_checker(price,maxv,lim)==0:
                        maxv=price
                       
                        continue
                    elif distance_checker(price,maxv,lim)==1:
                        # print('i am here 4')
                        distcounter+=1
                        if find_status(data,i,window,c_size)==1:
                            # print('HEREEE')
                            # print(ind)
                            trade=-1
                            openprice=price
                            opendate=val_h['time']
                            lowp=val_h['Low']
                            # print('i am here ')
                            sl=lowp-sl_adj
                            expectedloss=price-sl
                            target_tp=expectedloss*multiplier
                            tp=price+target_tp
                            # print(trade)
                            statuscounter+=1
                        # elif find_status(data,i,window,c_size)==0:
                            
                            # continue
                            
                elif price<minv:
                    if distance_checker(price,minv,lim)==0:
                        minv=price
                       
                        
                        continue
                    elif distance_checker(price,minv,lim)==1:
                        # print('i am here3 ')
                        distcounter2+=1
                        if find_status(data,i,window,c_size)==1:
                            # print('found me 1')
                            trade=1
                            openprice=price
                            # print(trade)
                            opendate=val_h['time']
                            # print('i am here2 ')
                            highp=val_h['High']
                            sl=highp-sl_adj
                            expectedloss=sl-price
                            target_tp=expectedloss*multiplier
                            tp=price-target_tp
                            statuscounter2+=1
                        # elif find_status(data,i,window,c_size)==0:
                            
                            # continue
            elif trade==1:
                # print('i am here8 ')
                if price>tp:
                    # print('i am here 6')
                    closedate=val_h['time'] 
                    profit=calc_prof(price,openprice)
                    cash=cash+profit
                    print(cash)
                    if cash<=0:
                        print("CASH OUT ")
                        break
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
                elif price<sl:
                    # print('i am here7 ')
                    closedate=val_h['time']
                    profit=calc_prof(price,openprice)
                    cash=cash+profit
                    print(cash)
                    if cash<=0:
                        print("CASH OUT ")
                        break
                    trades.append(('Buy',opendate,closedate,profit))
                    
                    trade=0
            elif trade==-1:
                # print('i am here 9 ')
                if price<tp:
                    # print('i am here 10 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    cash=cash+profit
                    print(cash)
                    if cash<=0:
                        print("CASH OUT ")
                        break
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
                elif price>sl:
                    # print('i am here 11 ')
                    closedate=val_h['time']
                    profit=calc_prof(openprice,price)
                    cash=cash+profit
                    print(cash)
                    if cash<=0:
                        print("CASH OUT ")
                        break
                    trades.append(('Sell',opendate,closedate,profit))
                    
                    trade=0
    return trades,cash#,distcounter,statuscounter,distcounter2,statuscounter2
                    

In [12]:
ty,uy=daily(nowd,'03:00:00',1.5,10,1.5,1.5,5,20)

19.710000000000036
19.019999999999982
18.889999999999873
19.539999999999964
20.970000000000027
22.860000000000127
22.580000000000155
23.1400000000001
22.15000000000009
17.350000000000136
18.740000000000236
22.000000000000227
22.470000000000255
18.740000000000236
20.65000000000032
21.79000000000042
23.580000000000382
25.530000000000427
25.15000000000032
25.01000000000022
22.550000000000182
21.5300000000002
21.62000000000012
15.100000000000136
20.90000000000009
25.800000000000182
29.330000000000155
27.66000000000031
26.41000000000031
26.93000000000029
28.16000000000031
28.870000000000346
28.32000000000039
26.0600000000004
26.44000000000051
25.57000000000039
25.770000000000437
25.830000000000382
25.830000000000382
24.690000000000282
22.9200000000003
22.43000000000029
17.9200000000003
18.6700000000003
17.220000000000255
17.560000000000173
20.070000000000164
19.610000000000127
21.329999999999927
19.7199999999998
23.009999999999764
22.999999999999773
17.56999999999971
19.259999999999764
21.2

In [13]:
print(uy)

275.6800000000087
